In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#functions used for preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data_fake = pd.read_csv('/content/drive/MyDrive/Datasets/fake news ddataset/ISOT fake news dataset/Fake.csv')
data_true = pd.read_csv('/content/drive/MyDrive/Datasets/fake news ddataset/ISOT fake news dataset/True.csv')

In [ ]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
data_fake["class"] = 0
data_true["class"] = 1

In [ ]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

perorming manual testing for quality assesment , validation
data balencing, model evaluation


In [ ]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis = 0, inplace = True)

removed last 10 rows from the datasert and used them for manual testing


In [ ]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [1]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

In [ ]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [ ]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


concatinating the two dataframes, axis=0 i.e concat along the rows

In [ ]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [ ]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [ ]:
data.isnull().sum()

text     0
class    0
dtype: int64

shuffling the rows. frac=1 indicates that we are using all the rows for shuffling

In [ ]:
data = data.sample(frac = 1)

In [ ]:
data.head()

,text,class
14936,"WAKE UP AMERICA! Muslim terrorists, refugee...",0
7168,WASHINGTON (Reuters) - U.S. Republican Preside...,1
22489,21st Century Wire says If you have only recent...,0
9258,The disgraced Representative John Conyers (D-M...,0
9888,WASHINGTON (Reuters) - President Barack Obama ...,1


In [ ]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [ ]:
data.columns

Index(['text', 'class'], dtype='object')

In [ ]:
data.head()

,text,class
0,"WAKE UP AMERICA! Muslim terrorists, refugee...",0
1,WASHINGTON (Reuters) - U.S. Republican Preside...,1
2,21st Century Wire says If you have only recent...,0
3,The disgraced Representative John Conyers (D-M...,0
4,WASHINGTON (Reuters) - President Barack Obama ...,1


In [ ]:
# def wordopt(text):
#     text = text.lower()
#     text = re.sub('\[.*?\]','', text) #remove text enclosed in square brackets, including the brackets themselves
#     text = re.sub("\\W", " ", text) #replaces all non-word characters (e.g., punctuation, special characters, symbols) with spaces
#     text = re.sub('https?://\S+|www\.\S+', '', text) # removes URLs from the text by matching and removing both HTTP/HTTPS URLs and "www" URLs
#     text = re.sub('<.*?>+', '', text) #remove HTML tags and their contents
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove all punctuation characters
#     text = re.sub('\n', '', text) #removes newline characters, which are typically used to represent line breaks or paragraphs in text
#     text = re.sub('\w*\d\w', '', text) #removes words containing numbers or alphanumeric patterns
#     return text

In [ ]:
def wordopt(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove text enclosed in square brackets, including the brackets themselves
    text = re.sub('\[.*?\]', '', text)

    # Remove non-word characters (punctuation, special characters, symbols)
    text = re.sub(r'\W', ' ', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags and their contents
    text = re.sub(r'<.*?>', '', text)

    # Remove all punctuation characters
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)

    # Remove newline characters
    text = re.sub(r'\n', ' ', text)

    # Remove words containing numbers or alphanumeric patterns
    text = re.sub(r'\w*\d\w*', '', text)

    # Tokenization (split text into words)
    tokens = text.split()

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization, Lemmatization tries to find the base or dictionary form of a word
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the cleaned and preprocessed tokens back into a single text
    text = ' '.join(tokens)

    return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

spitting the data into training data nd testing data


In [ ]:
x = data['text']
y = data['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25,random_state=42)

performing word embedding


downloading the 200D GloVe vectors

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-10-28 13:21:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-28 13:21:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-28 13:21:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov


load_glove_vectors function takes the path to a GloVe vectors file as input (e.g., 'glove.6B.200d.txt').

using 50d as the system is getting crashed



In [ ]:
def load_glove_vectors(glove_file):
    word_vectors = {} # Create an empty dictionary to store word vectors
    with open(glove_file, 'r', encoding='utf-8') as file:     #open the GloVe file in read mode ('r') with the 'utf-8' encoding.
        for line in file:     # each line in the file
            values = line.split()   #split the line into a list of values using spaces, so you have the word and its vector values as separate items.
            word = values[0]  # The first value on each line is the word which is extracted
            vector = np.array(values[1:], dtype='float32')  # The rest are the vector values which are converted into a NumPy array of 32-bit floating-point numbers
            word_vectors[word] = vector    # Store the word and its vector in the dictionary
    return word_vectors   # Return the dictionary with word vectors

glove_file = '/content/glove.6B.50d.txt'  # Specify the path to the 200D GloVe vectors file
word_vectors = load_glove_vectors(glove_file)   # Call the function to load the vectors

glove_file variable is set to the path '/content/glove.6B.50d.txt', which is where your 200-dimensional GloVe vectors file is located

Map words in your preprocessed text data to the GloVe vectors:

tokenized_data represents your text data that has been split into words or tokens.

word_vectors is a dictionary that contains GloVe word vectors for various words.

In [ ]:
# Function to get GloVe word vector for a single word
def get_word_vector(word, word_vectors):
    if word in word_vectors:
        return word_vectors[word]
    else:
        # Handle the case when the word is not found in the word vectors
        # You can choose to return a random vector or any other suitable vector
        random_vector = np.random.rand(word_vectors['example'].shape[0])  # Generate a random vector
        return random_vector  # Return the random vector



In [ ]:
# Function to map tokens to GloVe word vectors
def map_tokens_to_vectors(text, word_vectors):
    data_vectors = []  # Initialize a list to store the data vectors
    for tokens in text:  # Loop through each document's tokens
        token_vectors = [get_word_vector(word, word_vectors) for word in tokens]  # Map each token to GloVe vector
        data_vectors.append(token_vectors)  # Add the list of token vectors to the data_vectors list
    return data_vectors  # Return the list of data vectors


In [ ]:

# Map training data to GloVe vectors
x_train_vectors = map_tokens_to_vectors(x_train, word_vectors)


In [ ]:

# Map testing data to GloVe vectors
x_test_vectors = map_tokens_to_vectors(x_test, word_vectors)


build and train a Bidirectional LSTM model using the GloVe word vectors:

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense

In [ ]:
# Define the maximum sequence length
max_seq_length = 1000

In [ ]:
# Create a function to pad sequences to a fixed length
def pad_sequences(data, max_length):
    return [item[:max_length] if len(item) > max_length else item + [np.zeros(data[0][0].shape)] * (max_length - len(item)) for item in data]


In [ ]:
# Pad the sequences to a fixed length
x_train_vectors_padded = pad_sequences(x_train_vectors, max_seq_length)
x_test_vectors_padded = pad_sequences(x_test_vectors, max_seq_length)


In [ ]:
# Create a Bidirectional LSTM model
model = Sequential()

# Add an Embedding layer with the pre-trained GloVe word vectors
model.add(Embedding(input_dim=len(word_vectors), output_dim=50, input_length=max_seq_length, weights=[x_train_vectors], trainable=True))

# Add a Bidirectional LSTM layer with 64 units
model.add(Bidirectional(LSTM(64, return_sequences=True)))

# Add a Dense layer with a single output unit and sigmoid activation (for binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


ValueError: ignored

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the Residual Block
def residual_block(x, filters, kernel_size=3):
    # Shortcut
    shortcut = x

    # First convolution
    x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu')(x)

    # Second convolution
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)

    # Add shortcut to the output
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x

# Define the ResNet model
def build_resnet(input_shape, num_classes):
    input_layer = layers.Input(shape=input_shape)
    x = input_layer

    # Initial Convolution
    x = layers.Conv1D(64, 7, padding='same', activation='relu')(x)

    # Residual Blocks
    for _ in range(3):  # You can adjust the number of residual blocks
        x = residual_block(x, 64)

    # Global Average Pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Fully connected layer
    x = layers.Dense(128, activation='relu')(x)

    # Output layer
    output = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs=input_layer, outputs=output)
    return model

# Define your input shape and the number of classes
input_shape = (max_seq_length, 50)  # Adjust input shape based on your data
num_classes = 2  # Number of classes in your classification task

# Build the ResNet model
model = build_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 50)]           0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 1000, 64)             22464     ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 1000, 64)             12352     ['conv1d[0][0]']              
                                                                                                  
 conv1d_2 (Conv1D)           (None, 1000, 64)             12352     ['conv1d_1[0][0]']            
                                                                                              

In [ ]:

# Train your model on the training data
model.fit(x_train_vectors, y_train, batch_size=64, epochs=10)



In [ ]:
# You can also evaluate the model on your testing data
test_loss, test_accuracy = model.evaluate(x_test_vectors, y_test)
print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.9850267379679144

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5872
           1       0.98      0.99      0.98      5348

    accuracy                           0.99     11220
   macro avg       0.98      0.99      0.98     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9954545454545455

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5872
           1       1.00      1.00      1.00      5348

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

0.9949197860962566

In [ ]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5906
           1       0.99      1.00      0.99      5314

    accuracy                           0.99     11220
   macro avg       0.99      1.00      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

0.988680926916221

In [ ]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5906
           1       0.99      0.99      0.99      5314

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not a Fake News"


In [ ]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                           output_lable(pred_DT[0]),
                                                                                                           output_lable(pred_GB[0]),
                                                                                                           output_lable(pred_RF[0])))

In [ ]:
news = str(input())
manual_testing(news)

 Right now, the whole world is looking at the shocking fact that Democrat Doug Jones beat Republican Roy Moore in the special election to replace Attorney General Jeff Sessions in the United States Senate. Of course, Moore s candidacy was rocked by allegations of sexually harassing and even molesting teenage girls   and being banned from the mall in his hometown of Gadsden, Alabama for doing so.Even before that, Moore was an incendiary character in Alabama politics, having been removed as Chief Justice of the Alabama Supreme Court not once but twice, and having made statements such as Muslims should not be allowed in Congress and homosexuality should be illegal. Hell, he even said that the last time America was great was when we had slavery. Therefore, he was an extraordinarily damaged candidate as it was. However, despite all of this, Alabama is a deep red state, with many voters agreeing with some of Moore s more extreme positions, and some even insisting that the allegations of sexu



LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news = str(input())
manual_testing(news)

 WASHINGTON (Reuters) - The U.S. Senate will vote on final tax legislation on Tuesday evening, Senate Republican leader Mitch McConnell said, potentially allowing President Donald Trump to sign the bill into law as early as Wednesday. â€œCongress is standing at the doorstep of a historic opportunity,â€ McConnell said on the floor of the Senate as he announced the voteâ€™s timing. The House of Representatives is scheduled to vote on the same legislation at 1:30 p.m. EST (1830 GMT). If both chambers of Congress pass the bill, Trump will be able to meet his goal of signing it into law before Christmas. 




LR Prediction: Not a Fake News 
DT Prediction: Not a Fake News 
GB Prediction: Not a Fake News 
RF Prediction: Not a Fake News
